In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
path = '/data/examples/ibmxfet/'

file_dir = os.listdir(path)

In [3]:
file_dir

['profile.txt',
 'training.csv',
 'testing.csv',
 'behavior_sim.csv',
 'network_sim.csv',
 'usage_data.txt']

In [4]:
profile = pd.read_csv(path+file_dir[0])
training = pd.read_csv(path+file_dir[1])
testing = pd.read_csv(path+file_dir[2])
behavior = pd.read_csv(path+file_dir[3])
network_sim = pd.read_csv(path+file_dir[4])
usage_data = pd.read_csv(path+file_dir[5])

In [6]:
profile.head()

,USER_ID,SRV_TYPE_DESC,SUBSCR_STATUS_CODE,GENDER_CODE,P6M_AVG_NET_INV_AMT,AGE,TENURE,BILL_CITY_NAME,BILL_DISTRICT_NAME,SMS_PROM_IND,IMEI_TYPE,IMEI_MFG_NAME,IMEI_MKT_NAME,IMEI_SMART_OS_FLAG,Credit_IND,AGENCY_FLAG,Third_Party_Payment_IND,FLAT_RATE_4G_IND,P6M_AVG_DATA_USAGE_MB
0,1,4G,A,M,600~800,25~29,66,臺中市,北區,Y,Smart Phone,HTC,10,Android 6.0,N,N,N,Y,4466
1,2,4G,A,F,600~800,75~79,12,新北市,中和,N,Smart Phone,OPPO,R9s,Android 6.0,N,N,N,Y,4681
2,3,4G,A,F,1000~1200,35~39,39,宜蘭縣,五結,Y,Smart Phone,HTC,Desire 830,Android 5.1,Y,Y,Y,Y,4138
3,4,4G,A,M,1200~1400,40~44,21,臺北市,大安區,Y,Smart Phone,Apple,iPhone 6 Plus,iOS 8,Y,N,N,Y,21216
4,6,4G,A,F,1600~1800,25~29,55,嘉義縣,中埔,Y,Smart Phone,Apple,iPhone 6,iOS 8,N,N,N,Y,28655


In [40]:
le = LabelEncoder()

In [41]:
le.fit(profile['P6M_AVG_NET_INV_AMT'])
le.classes_
cost = le.transform(profile['P6M_AVG_NET_INV_AMT'])


In [42]:
le.fit(profile['AGE'])
le.classes_
age = le.transform(profile['AGE'])

In [43]:
le.fit(profile['TENURE'])
le.classes_
tenure = le.transform(profile['TENURE'])

In [44]:
le.fit(profile['IMEI_TYPE'])
le.classes_
device = le.transform(profile['IMEI_TYPE'])

In [45]:
le.fit(profile['IMEI_MFG_NAME'])
le.classes_
brand = le.transform(profile['IMEI_MFG_NAME'])

In [46]:
le.fit(profile['IMEI_MKT_NAME'])
le.classes_
brand_model = le.transform(profile['IMEI_MKT_NAME'])

In [47]:
gender = pd.get_dummies(profile['GENDER_CODE'])
city = pd.get_dummies(profile['BILL_CITY_NAME'])
sms = pd.get_dummies(profile['SMS_PROM_IND'])
credit = pd.get_dummies(profile['Credit_IND'])
agency = pd.get_dummies(profile['AGENCY_FLAG'])
_3rdpay = pd.get_dummies(profile['Third_Party_Payment_IND'])
_4gfull = pd.get_dummies(profile['FLAT_RATE_4G_IND'])
avg_usage = profile['P6M_AVG_DATA_USAGE_MB']


In [48]:
temp = pd.DataFrame()
append = temp.append([cost,age,tenure,device,brand,brand_model])
append = append.transpose()
append.columns = ['cost','age','tenure','device','brand','brand_model']


In [49]:
encoded_feature = pd.concat([profile['USER_ID'],gender,city,sms,credit,agency,_3rdpay,
                             _4gfull,avg_usage,append],axis=1)
encoded_feature.head()

,USER_ID,F,M,南投縣,嘉義市,嘉義縣,基隆市,宜蘭縣,屏東縣,彰化縣,...,Y,N,Y,P6M_AVG_DATA_USAGE_MB,cost,age,tenure,device,brand,brand_model
0,1,0,1,0,0,0,0,0,0,0,...,0,0,1,4466,17,1,64,2,7,2
1,2,1,0,0,0,0,0,0,0,0,...,0,0,1,4681,17,11,10,2,16,141
2,3,1,0,0,0,0,0,1,0,0,...,1,0,1,4138,1,3,37,2,7,34
3,4,0,1,0,0,0,0,0,0,0,...,0,0,1,21216,2,4,19,2,2,225
4,6,1,0,0,0,1,0,0,0,0,...,0,0,1,28655,4,1,53,2,2,224


In [50]:
usage_sum = usage_data.groupby('USER_ID',as_index=False).sum()
usage_sum.head()

,USER_ID,MO_CALL_DUR,MT_CALL_DUR,DATA_USG_MB
0,1,2412,6640,2787.69
1,2,741,866,3147.82
2,3,393,51,5376.46
3,4,3207,6616,12812.16
4,6,769,67,29411.16


In [51]:
with open('usage_sum','wb') as file:
    pickle.dump(usage_sum,file)

NameError: name 'pickle' is not defined

In [ ]:
call_out_usage = []
for i in range(len(usage_sum)):
    if usage_sum['MO_CALL_DUR'][i] < 10:
        call_out_usage.append(1)
    elif (usage_sum['MO_CALL_DUR'][i]>10)&(usage_sum['MO_CALL_DUR'][i]<60):
        call_out_usage.append(2)
    elif (usage_sum['MO_CALL_DUR'][i]>60)&(usage_sum['MO_CALL_DUR'][i]<180):
        call_out_usage.append(3)
    elif (usage_sum['MO_CALL_DUR'][i]>180)&(usage_sum['MO_CALL_DUR'][i]<300):
        call_out_usage.append(4)
    elif (usage_sum['MO_CALL_DUR'][i]>300)&(usage_sum['MO_CALL_DUR'][i]<500):
        call_out_usage.append(5)
    elif (usage_sum['MO_CALL_DUR'][i]>500)&(usage_sum['MO_CALL_DUR'][i]<750):
        call_out_usage.append(6)
    elif (usage_sum['MO_CALL_DUR'][i]>750)&(usage_sum['MO_CALL_DUR'][i]<1000):
        call_out_usage.append(7)
    elif (usage_sum['MO_CALL_DUR'][i]>1000)&(usage_sum['MO_CALL_DUR'][i]<1400):
        call_out_usage.append(8)
    elif (usage_sum['MO_CALL_DUR'][i]>1400)&(usage_sum['MO_CALL_DUR'][i]<2000):
        call_out_usage.append(9)
    elif (usage_sum['MO_CALL_DUR'][i]>2000)&(usage_sum['MO_CALL_DUR'][i]<4000):
        call_out_usage.append(10)
    else:
        call_out_usage.append(11)

In [52]:
call_in_usage = []
for i in range(len(usage_sum)):
    if usage_sum['MT_CALL_DUR'][i] < 20:
        call_in_usage.append(1)
    elif (usage_sum['MT_CALL_DUR'][i]>20)&(usage_sum['MT_CALL_DUR'][i]<90):
        call_in_usage.append(2)
    elif (usage_sum['MT_CALL_DUR'][i]>90)&(usage_sum['MT_CALL_DUR'][i]<180):
        call_in_usage.append(3)
    elif (usage_sum['MT_CALL_DUR'][i]>180)&(usage_sum['MT_CALL_DUR'][i]<320):
        call_in_usage.append(4)
    elif (usage_sum['MT_CALL_DUR'][i]>320)&(usage_sum['MT_CALL_DUR'][i]<550):
        call_in_usage.append(5)
    elif (usage_sum['MT_CALL_DUR'][i]>550)&(usage_sum['MT_CALL_DUR'][i]<800):
        call_in_usage.append(6)
    elif (usage_sum['MT_CALL_DUR'][i]>800)&(usage_sum['MT_CALL_DUR'][i]<1100):
        call_in_usage.append(7)
    elif (usage_sum['MT_CALL_DUR'][i]>1100)&(usage_sum['MT_CALL_DUR'][i]<1600):
        call_in_usage.append(8)
    elif (usage_sum['MT_CALL_DUR'][i]>1600)&(usage_sum['MT_CALL_DUR'][i]<2400):
        call_in_usage.append(9)
    elif (usage_sum['MT_CALL_DUR'][i]>2400)&(usage_sum['MT_CALL_DUR'][i]<4000):
        call_in_usage.append(10)
    else:
        call_in_usage.append(11)

In [53]:
data_usage = []
for i in range(len(usage_sum)):
    if usage_sum['DATA_USG_MB'][i] < 400:
        data_usage.append(1)
    elif (usage_sum['DATA_USG_MB'][i]>400)&(usage_sum['DATA_USG_MB'][i]<1000):
        data_usage.append(2)
    elif (usage_sum['DATA_USG_MB'][i]>1000)&(usage_sum['DATA_USG_MB'][i]<2000):
        data_usage.append(3)
    elif (usage_sum['DATA_USG_MB'][i]>2000)&(usage_sum['DATA_USG_MB'][i]<3200):
        data_usage.append(4)
    elif (usage_sum['DATA_USG_MB'][i]>3200)&(usage_sum['DATA_USG_MB'][i]<5000):
        data_usage.append(5)
    elif (usage_sum['DATA_USG_MB'][i]>5000)&(usage_sum['DATA_USG_MB'][i]<7200):
        data_usage.append(6)
    elif (usage_sum['DATA_USG_MB'][i]>7200)&(usage_sum['DATA_USG_MB'][i]<10000):
        data_usage.append(7)
    elif (usage_sum['DATA_USG_MB'][i]>10000)&(usage_sum['DATA_USG_MB'][i]<14000):
        data_usage.append(8)
    elif (usage_sum['DATA_USG_MB'][i]>14000)&(usage_sum['DATA_USG_MB'][i]<24000):
        data_usage.append(9)
    else:
        data_usage.append(10)

In [54]:
temp = pd.DataFrame()
append = temp.append([call_out_usage,call_in_usage,data_usage])
append = append.transpose()
append.columns = ['call_out_usage','call_in_usage','data_usage']


In [55]:
encoded_feature = pd.concat([encoded_feature,append],axis=1)
encoded_feature.head()

,USER_ID,F,M,南投縣,嘉義市,嘉義縣,基隆市,宜蘭縣,屏東縣,彰化縣,...,P6M_AVG_DATA_USAGE_MB,cost,age,tenure,device,brand,brand_model,call_out_usage,call_in_usage,data_usage
0,1,0,1,0,0,0,0,0,0,0,...,4466,17,1,64,2,7,2,10,11,4
1,2,1,0,0,0,0,0,0,0,0,...,4681,17,11,10,2,16,141,6,7,4
2,3,1,0,0,0,0,0,1,0,0,...,4138,1,3,37,2,7,34,5,2,6
3,4,0,1,0,0,0,0,0,0,0,...,21216,2,4,19,2,2,225,10,11,8
4,6,1,0,0,0,1,0,0,0,0,...,28655,4,1,53,2,2,224,7,2,10


In [56]:
lis = []
for name in range(len(encoded_feature.columns)):
    d = str(encoded_feature.columns[name]).replace('<','')
    lis.append(d)

In [57]:
encoded_feature.columns = lis

In [58]:
encoded_feature.head()

,USER_ID,F,M,南投縣,嘉義市,嘉義縣,基隆市,宜蘭縣,屏東縣,彰化縣,...,P6M_AVG_DATA_USAGE_MB,cost,age,tenure,device,brand,brand_model,call_out_usage,call_in_usage,data_usage
0,1,0,1,0,0,0,0,0,0,0,...,4466,17,1,64,2,7,2,10,11,4
1,2,1,0,0,0,0,0,0,0,0,...,4681,17,11,10,2,16,141,6,7,4
2,3,1,0,0,0,0,0,1,0,0,...,4138,1,3,37,2,7,34,5,2,6
3,4,0,1,0,0,0,0,0,0,0,...,21216,2,4,19,2,2,225,10,11,8
4,6,1,0,0,0,1,0,0,0,0,...,28655,4,1,53,2,2,224,7,2,10


In [60]:
training[training['USER_ID']==1]

,USER_ID,date,y
0,1,2018-03-03,0.488242
1,1,2018-03-04,0.481169
2,1,2018-03-05,0.015337
3,1,2018-03-06,0.022330
4,1,2018-03-07,0.019055
5,1,2018-03-08,0.017952
6,1,2018-03-09,0.019987
7,1,2018-03-10,0.431245
8,1,2018-03-11,0.046564
9,1,2018-03-12,0.460840


In [61]:
d = {}
for i in range(3001):
    d[i] = pd.DataFrame(training[training['USER_ID']==i])

In [62]:
for key in d:
    d[key] = d[key].reset_index()

In [63]:
df = pd.concat([d[key] for key in d], axis=1)
df = df.drop(['index','USER_ID'],axis=1)
df = df.dropna(axis=1)
df['DATE'] = df.iloc[:,0]
df = df.drop('date', axis=1)
df = df.transpose()
df.index = range(2629)
df.columns = df.iloc[2628]
df = df.reindex(df.index.drop(2628))
df.index = range(2628)

In [64]:
# # weekday: 週一~週五 weekend: 週六~週日
# weekday1 = df.iloc[:,2:7]
# weekday2 = df.iloc[:,9:13]
# weekday = pd.concat([weekday1,weekday2],axis=1)

# weekend1 = df.iloc[:,:2]
# weekend2 = df.iloc[:,7:9]
# weekend = pd.concat([weekend1, weekend2],axis=1)

# weekday.columns = ['2018-03-05','2018-03-06','2018-03-07','2018-03-08','2018-03-09','2018-03-12','2018-03-13','2018-03-14','2018-03-15']
# weekend.columns = ['2018-03-03','2018-03-04','2018-03-10','2018-03-11']

In [76]:
week1 = df.iloc[:,:6]
week2 = df.iloc[:,7:]

In [77]:
# # weekday: 週一~週四 weekend: 週五~週日
# weekday1 = df.iloc[:,2:6]
# weekday2 = df.iloc[:,9:13]
# weekday = pd.concat([weekday1,weekday2],axis=1)

# weekend1 = df.iloc[:,:2]
# weekend2 = df.iloc[:,6:9]
# weekend = pd.concat([weekend1, weekend2],axis=1)

# weekday.columns = ['2018-03-05','2018-03-06','2018-03-07','2018-03-08','2018-03-12','2018-03-13','2018-03-14','2018-03-15']
# weekend.columns = ['2018-03-03','2018-03-04','2018-03-09','2018-03-10','2018-03-11']

In [78]:
week1['med'] = week1.median(axis=1)
week2['med'] = week2.median(axis=1)
week1['mean'] = week1.mean(axis=1)
week2['mean'] = week2.mean(axis=1)
week1['max'] = week1.max(axis=1)
week2['max'] = week2.max(axis=1)
week1['min'] = week1.min(axis=1)
week2['min'] = week2.min(axis=1)
week1['25per'] = week1.quantile(.25,axis=1)
week2['25per'] = week2.quantile(.25,axis=1)
week1['75per'] = week1.quantile(.75,axis=1)
week2['75per'] = week2.quantile(.75,axis=1)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [80]:
week2

2628,2018-03-10,2018-03-11,2018-03-12,2018-03-13,2018-03-14,2018-03-15,med,mean,max,min,25per,75per
0,0.431245,0.0465645,0.46084,0.0400624,0.18811,0.0211169,0.117337,0.186468,0.460840,0.021117,0.093282,0.186468
1,0.0187458,0.0764715,0.375583,0.0621104,0.0357951,0.203902,0.069291,0.120271,0.375583,0.018746,0.056655,0.120271
2,0.0858309,0.0165351,0.138067,0.0884435,0.100995,0.365496,0.094719,0.127155,0.365496,0.016535,0.075173,0.127155
3,0.212154,0.0144216,0.113285,0.111387,0.0906392,0.078444,0.101013,0.103049,0.212154,0.014422,0.079365,0.103049
4,0.484287,0.12947,0.1511,0.15427,0.130922,0.0812661,0.141011,0.181761,0.484287,0.081266,0.126075,0.181761
5,0.077939,0.0655344,0.0625214,0.0652122,0.063273,0.0671887,0.065373,0.066720,0.077939,0.062521,0.064660,0.066720
6,0.00754007,0.00390879,0.00848387,0.22047,0.318307,0.376553,0.114477,0.149963,0.376553,0.003909,0.086835,0.149963
7,0.0417363,0.374435,0.00624949,0.04245,0.0426876,0.0421975,0.042324,0.084583,0.374435,0.006249,0.033305,0.084583
8,0.120017,1.08271,0.0725186,1.02885,0.0959698,0.137738,0.128877,0.380954,1.082708,0.072519,0.114787,0.380954
9,0.0260906,0.00450627,0.023631,0.0249797,0.0271835,0.025706,0.025343,0.022491,0.027184,0.004506,0.017995,0.025343


In [81]:
encoded_feature.to_csv('encoded_feature.csv',index=None)
week1.to_csv('week1.csv',index=None)
week2.to_csv('week2.csv',index=None)
df.to_csv('date.csv',index=None)